In [1]:
# https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter?resource=download

In [2]:
import pandas as pd 
from collections import defaultdict
import json

def extract_data_from_json(json_format):
    json_file = json.loads(json_format)
    dialog, summary = json_file['dialog'], json_file['summaries']
    conversation_id = dialog['dialog_id']
    
    full_dialog = ""
    for turn in dialog['turns']:
        is_agent, sentences = turn['is_agent'], turn['sentences']
        speech = "Agent: " if is_agent else "Customer: " + ' '.join(sentences) + '\n'
        full_dialog += speech
    
    extractive_summary = defaultdict(lambda: "")
    for i, item in enumerate(summary['extractive_summaries']):
        item = item[0]
        is_agent, sentences = item['is_agent'], item['sentences']
        speech = "Agent: " if is_agent else "Customer: " + ' '.join(sentences) + '\n'
        extractive_summary[i] = speech
    
    abstractive_summary = defaultdict(lambda: "")
    for i, sentences in enumerate(summary['abstractive_summaries']):
        abstractive_summary[i] = ' '.join(sentences)
    
    return full_dialog, extractive_summary, abstractive_summary

from tweet_sum_processor import TweetSumProcessor
import os 
TWCS_FILE_PATH = 'twcs/twcs.csv'

!mkdir tweetsum/
for split in os.listdir('tweet_sum_data_files/'):
    TWEET_SUMM_FILE_PATH = os.path.join('tweet_sum_data_files/', split)
    processor = TweetSumProcessor(TWCS_FILE_PATH)
    out = defaultdict(lambda: [])
    with open(TWEET_SUMM_FILE_PATH) as f:
        dialog_with_summaries = processor.get_dialog_with_summaries(f.readlines())
        for dialog_with_summary in dialog_with_summaries:
            try:
                json_format = dialog_with_summary.get_json()
            except Exception as e:
                print (e)
                print (dialog_with_summary)

            full_dialog, extractive_summary, abstractive_summary = extract_data_from_json(json_format)
            out['full_dialog'].append(full_dialog)
            for i in range(5):
                out[f'extractive_summary_{i}'].append(extractive_summary[i])
            for i in range(5):
                out[f'abstractive_summary_{i}'].append(abstractive_summary[i])

        out = pd.DataFrame.from_dict(out)
        data_split = split.split('_')[1]
        print (data_split)
        out.to_excel(f'tweetsum/{data_split}.xlsx', index=False)

mkdir: tweetsum/: File exists
valid
test
train
